In [55]:
import pandas as pd
import numpy as np
import states_abbrevs
from states_abbrevs import dict2
import glob

In [36]:
path = r"C:\Users\Steven Lantigua\Desktop\7ParkCP\vacancydata" 
files = glob.glob(path + "/*.csv")
years = ['2013','2014','2015','2016','2017','2018','2019']
li=[]

for (f,y) in zip(files,years):
    df = pd.read_csv(f, index_col=None, header=0)
    df = df.loc[1:] #removes unnecessary column description
    df['Year'] = y
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
framec = frame.copy()

In [37]:
framec.rename(columns={
    'DP04_0001E':'Total Housing Units','DP04_0002E':'Occupied Units', 'DP04_0003E':'Vacant Units',
    'DP04_0004E':'Homeowner Vacancy Rate','DP04_0005E':'Rental Vacancy Rate','NAME':'County'}, inplace=True)

In [38]:
framec.columns

Index(['GEO_ID', 'County', 'Total Housing Units', 'DP04_0001M', 'DP04_0001PE',
       'DP04_0001PM', 'Occupied Units', 'DP04_0002M', 'DP04_0002PE',
       'DP04_0002PM',
       ...
       'DP04_0141PM', 'Year', 'DP04_0142E', 'DP04_0142M', 'DP04_0142PE',
       'DP04_0142PM', 'DP04_0143E', 'DP04_0143M', 'DP04_0143PE',
       'DP04_0143PM'],
      dtype='object', length=575)

In [39]:
vacantdf = framec.loc[:,['Year','County','Total Housing Units','Occupied Units','Vacant Units','Rental Vacancy Rate','Homeowner Vacancy Rate']]

## fixing 'County' format

In [40]:
vacantdf['county']=vacantdf['County'].str.split(',',1,expand=True)[0]
vacantdf['state'] = vacantdf['County'].str.split(',',1,expand=True)[1]
vacantdf['state'] = vacantdf['state'].apply(lambda x: x.strip())
vacantdf['state'] = vacantdf['state'].map(dict2)
vacantdf['County'] = vacantdf['county'] + ', ' + vacantdf['state']
vacantdf.drop('county',axis=1,inplace=True)

In [41]:
vacantdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5832 entries, 0 to 5831
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Year                    5832 non-null   object
 1   County                  5832 non-null   object
 2   Total Housing Units     5832 non-null   object
 3   Occupied Units          5832 non-null   object
 4   Vacant Units            5832 non-null   object
 5   Rental Vacancy Rate     5832 non-null   object
 6   Homeowner Vacancy Rate  5832 non-null   object
 7   state                   5832 non-null   object
dtypes: object(8)
memory usage: 364.6+ KB


### Rental Vacancy Rate small sample size issue

The Rental vacancy rate value for some of the counties is an 'N' which
indicates that data for this geographic area cannot be displayed because the number of sample cases is too small
so I decided to drop them *(21 rows dropped)*

In [44]:
vacantdf = vacantdf[vacantdf['Rental Vacancy Rate'] != 'N']

### Changing column data to appropriate Dtype

In [49]:
vacantdf['Year'] = pd.to_datetime(vacantdf['Year'], format = '%Y').dt.year
vacantdf['Total Housing Units'] = vacantdf['Total Housing Units'].apply(lambda x: int(x))
vacantdf['Occupied Units'] = vacantdf['Occupied Units'].apply(lambda x: int(x))
vacantdf['Vacant Units'] = vacantdf['Vacant Units'].apply(lambda x: int(x))
vacantdf['Rental Vacancy Rate'] = vacantdf['Rental Vacancy Rate'].apply(lambda x: float(x))
vacantdf['Homeowner Vacancy Rate'] = vacantdf['Homeowner Vacancy Rate'].apply(lambda x: float(x))

<ipython-input-49-6655b4c3d267>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacantdf['Year'] = pd.to_datetime(vacantdf['Year'], format = '%Y').dt.year
<ipython-input-49-6655b4c3d267>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacantdf['Total Housing Units'] = vacantdf['Total Housing Units'].apply(lambda x: int(x))
<ipython-input-49-6655b4c3d267>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [50]:
vacantdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5811 entries, 0 to 5831
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Year                    5811 non-null   int64  
 1   County                  5811 non-null   object 
 2   Total Housing Units     5811 non-null   int64  
 3   Occupied Units          5811 non-null   int64  
 4   Vacant Units            5811 non-null   int64  
 5   Rental Vacancy Rate     5811 non-null   float64
 6   Homeowner Vacancy Rate  5811 non-null   float64
 7   state                   5811 non-null   object 
dtypes: float64(2), int64(4), object(2)
memory usage: 408.6+ KB


In [58]:
new_vacant = vacantdf.loc[:,['Year','County','Rental Vacancy Rate']]

### Interpolating for month value

In [59]:
m_vacancy = pd.DataFrame()
for county, df in new_vacant.groupby('County'):
    df['rateChange'] = df['Rental Vacancy Rate'].diff() / df['Rental Vacancy Rate'].shift()
    df = df.iloc[np.arange(df.shape[0]).repeat(12)].reset_index().drop('index', axis = 1)
    df = pd.concat([df, pd.DataFrame(list(range(1, 13)) * int(df.shape[0] / 12), columns = ['Month'])], axis = 1)
    df = df[df['Year'] != 2013]
    for year in pd.unique(df['Year']):
        df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
    m_vacancy = pd.concat([m_vacancy, df], axis = 0)

<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['

<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['

<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['

<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['

<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['

...

<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['

<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['

<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['

<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['

<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['Year'] == 2014]['Month'])
<ipython-input-59-25a6fabac5b3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df['Year'] == year]['Rental Vacancy Rate'] = df[df['Year'] == 2014]['Rental Vacancy Rate'] * (((1 + df[df['Year'] == 2014]['rateChange']) ** (1/12)) ** df[df['

In [60]:
m_vacancy

,Year,County,Rental Vacancy Rate,rateChange,Month
12,2014,"Ada County, ID",3.1,-0.114286,1
13,2014,"Ada County, ID",3.1,-0.114286,2
14,2014,"Ada County, ID",3.1,-0.114286,3
15,2014,"Ada County, ID",3.1,-0.114286,4
16,2014,"Ada County, ID",3.1,-0.114286,5
...,...,...,...,...,...
79,2019,"Yuma County, AZ",5.1,-0.320000,8
80,2019,"Yuma County, AZ",5.1,-0.320000,9
81,2019,"Yuma County, AZ",5.1,-0.320000,10
82,2019,"Yuma County, AZ",5.1,-0.320000,11
